# Prepare driver

In [1]:
! wget -P /home/jovyan https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar

--2019-06-26 16:01:17--  https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.5/postgresql-42.2.5.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.60.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.60.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 825943 (807K) [application/java-archive]
Saving to: ‘/home/jovyan/postgresql-42.2.5.jar’

postgresql-42.2.5.j 100%[===================>] 806.58K  3.96MB/s    in 0.2s    

2019-06-26 16:01:18 (3.96 MB/s) - ‘/home/jovyan/postgresql-42.2.5.jar’ saved [825943/825943]



# Start spark application

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path /home/jovyan/postgresql-42.2.5.jar --jars /home/jovyan/postgresql-42.2.5.jar pyspark-shell'

import pyspark

spark = pyspark.sql.SparkSession.builder \
        .master("local[1]") \
        .appName("snapshot") \
        .getOrCreate()

# Warm-up spark

In [3]:
spark.sparkContext.range(1000).sum()

499500

# Read some PostgreSQL Data

In [7]:
customers = spark.read.format('jdbc').options(
        url = "jdbc:postgresql://postgres:5432/postgres?user=postgres&password=postgres&currentSchema=inventory",
        database='postgres',
        dbtable='customers'
    ).load()

customers.show()

+----+----------+---------+--------------------+
|  id|first_name|last_name|               email|
+----+----------+---------+--------------------+
|1004|      Anne|Kretchmar| toddevans@yahoo.com|
|1009|    Amanda|   Hardin|crystal60@larsen-...|
|1002|    George|   Bailey| iterrell@boyer.info|
|1010|      Tina| Valencia| fjordan@hotmail.com|
|1011|    Carlos|    Poole|kingjoshua@huerta...|
|1007|     David| Cisneros|hendersonchristop...|
|1003|    Edward|   Walker|xwhitehead@hotmai...|
|1013|    Amanda|    Evans|azimmerman@gmail.com|
|1015|   Richard|     Hale|sparrish@wallace.com|
|1016|   Michael|   Larson|lindsey64@hotmail...|
|1017|   Patrick| Williams|   zlawson@jones.com|
|1014|   Phillip|     Rice|nrodriguez@love-k...|
|1018|    Robert|    Watts|debbielowe@hotmai...|
|1020|  Nicholas| Hamilton|jessicahodge@cook...|
|1001|     Sally|   Thomas| umarshall@yahoo.com|
|1021|    Lauren| Harrison|    nlewis@yahoo.com|
+----+----------+---------+--------------------+



# Write snapshot to Data Lake

In [8]:
customers.write.format("parquet").save("/home/jovyan/customers_data")

In [9]:
spark.stop()